# xarray and netCDF files

The `xarray` Python package can be used to extract netCDF files which are often used to distribute meteorological data and normally end with the extension ".nc". More details of this package can be found at:

http://xarray.pydata.org/en/stable/

You won't need to go too much into detail of the features of `xarray` but it is useful to understand how to use it to read in files and how to extract data from those files.

### Installing xarray

Unlike `numpy`, which is available by default, you will need to make sure that `xarray` is available within your Anaconda installation if you haven't already.

- Within Anaconda Navigator there should be a tab labelled "Environments". This will show you a list of currently installed packages.
- In the drop down if you select "All" (rather than "Installed") you can then select `xarray` from the list of available packages by ticking it (you can search to find it) and clicking "Apply". This should then download xarray for you (may take a few minutes).

### Opening a netCDF file and reading data

As with all additional Python packages (like `numpy`) we start by importing the module. This time we're going to use the shorthand `xr` to access functions from the module:

In [1]:
import xarray as xr

We can now open a netCDF file and take a look. This contains similar data to the text file used in the previous chapter but now the file is in a different format and cannot be opened in a text editor.

In [2]:
filename = "data/AGAGE-GC-FID_MHD_19940101_ch4-10m.nc"

In [3]:
ds = xr.open_dataset(filename)

We have now created what's called a `Dataset`. This is basically a collection of data which has been nicely labelled.
Let's have a look at what a Dataset looks like:

In [4]:
print(ds)

<xarray.Dataset>
Dimensions:               (time: 264373)
Coordinates:
  * time                  (time) datetime64[ns] 1994-02-17T20:18:35 ... 2018-07-12T03:17:35
Data variables:
    ch4                   (time) float64 ...
    ch4_repeatability     (time) float64 ...
    ch4_status_flag       (time) int64 ...
    ch4_integration_flag  (time) int64 ...
Attributes:
    comment:              Gas chromatograph measurements. Output from GCWerks.
    Source:               In situ measurements of air
    Processed by:         Matt Rigby, University of Bristol (matt.rigby@brist...
    data_owner_email:     s.odoherty@bristol.ac.uk
    data_owner:           Simon O'Doherty
    inlet_height_magl:    10.0
    Conventions:          CF-1.6
    Conditions of use:    Ensure that you contact the data owner at the outse...
    File created:         2018-07-12 15:55:18.691414
    station_long_name:    Mace Head, Ireland
    station_height_masl:  5.0
    station_latitude:     53.32611
    station_longit

A dataset can contain up to four elements: Data Variables, Coordinates, Dimensions and Attributes.

You shouldn't worry too much about the details but basically:
 - **Data Variables** contain the data, in this case methane measurements, errors and data quality flags.
 - **Coordinates** give us a label for that data, in this case the time each of the measurements were taken.
 - **Dimensions** tell us how many points there are on each axis.
 - **Attributes** provide extra information about the data such as who processed it, when was the file created etc

To extract a particular data variable from the dataset we use its name as listed above. So for $\mathrm{CH_4}$ measurements we use the name "ch4". This can be done in one of two ways:

In [5]:
meas1 = ds.ch4
print(meas1)

<xarray.DataArray 'ch4' (time: 264373)>
[264373 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1994-02-17T20:18:35 ... 2018-07-12T03:17:35
Attributes:
    standard_name:         mole_fraction_of_methane_in_air
    long_name:             mole_fraction_of_methane_in_air
    units:                 1e-9
    ancilliary_variables:  ch4_repeatability ch4_status_flag ch4_integration_...


Accessing a data variable using this "dot syntax" has some downsides. For example, if the name clashes with an existing function name in a `Dataset` then this syntax will not work. You will see this style used in code but I recommend using the following syntax if possible:

In [6]:
meas2 = ds["ch4"]
print(meas2)

<xarray.DataArray 'ch4' (time: 264373)>
[264373 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1994-02-17T20:18:35 ... 2018-07-12T03:17:35
Attributes:
    standard_name:         mole_fraction_of_methane_in_air
    long_name:             mole_fraction_of_methane_in_air
    units:                 1e-9
    ancilliary_variables:  ch4_repeatability ch4_status_flag ch4_integration_...


You may notice that this output is a new object called a `DataArray`. It's very similar to a `Dataset` but it contains one piece of data rather than the several we can see in the `Dataset` above. `Dataset`s and `DataArray`s are very similar in their functionality.

More than this, if really we just want the actual values rather than another object then this can be extracted using the `values` attribute:

In [7]:
meas = ds['ch4'].values
print(meas)
print(type(meas))
print(len(meas))
print(meas.shape)

[1835.68  1840.116 1846.133 ... 1926.983 1917.464 1948.148]
<class 'numpy.ndarray'>
264373
(264373,)


As you can see when we asked what type the object was, this is just a numpy array and can be treated in the same way as we have seen previously e.g.:

In [1]:
print(ds['ch4'].values[0:5])
print(np.sum(ds['ch4'].values))

NameError: name 'ds' is not defined

You may see that the second command (containing `np.sum`) returned an error. If so, this is because we haven't actually told Python about the `numpy` module since the previous chapter so to get this to run we'll need to include an `import` statement (only once per document):

In [12]:
import numpy as np
print(np.sum(ds['ch4'].values))

497151687.12799984


Another useful thing that can be done is to look at the dimensions of your `DataArray`: 

In [ ]:
print(ds['ch4'].dims)

And we can extract the Coordinates associated with that label in the same way as we extracted the data variable earlier:

In [ ]:
time = ds['time']
print(time)

In [ ]:
print(time.values)

### Common Pitfalls

As `xarray.DataArray`s contain, and work similarly to `numpy.array`, it is easy to make the mistake of assuming they work exactly the same. Some functionality is subtly different from `numpy` and using a `DataArray` when an `array` is expected can lead to some confusing error messages. It can often be worthwhile trying to add a `.values` to your `DataArray` to see if this solves your problem. If this fixes the error you can either continue with your `array` or (preferably) you can look up the correct way to achieve the same result with the `DataArray`.

In [28]:
#create a simple 3D DataArray to demonstrate with
x = np.array([1,2,3])
y = np.array([1,2,3])
t = np.array([1])
XX, YY = np.meshgrid(x, y)
ZZ = np.expand_dims(XX * YY,axis=2)

example_array = xr.DataArray(ZZ, dims=["x","y","time"],
                             coords={"x":x,
                                    "y":y,
                                    "time":t})
print(example_array)

#try (incorrectly) to select only values where x is less than 2:
print(example_array[ (XX < 2),None])

<xarray.DataArray (x: 3, y: 3, time: 1)>
array([[[1],
        [2],
        [3]],

       [[2],
        [4],
        [6]],

       [[3],
        [6],
        [9]]])
Coordinates:
  * x        (x) int32 1 2 3
  * y        (y) int32 1 2 3
  * time     (time) int32 1


IndexError: Unlabeled multi-dimensional array cannot be used for indexing: x

In [32]:
#solving the problem with .values:
print(example_array.values[ (XX < 2),None])
#solving the problem with xarray functions:
print(example_array.where(example_array.x < 2, drop=True))
print(example_array.where(example_array.x < 2, drop=True).values)

[[[1]]

 [[2]]

 [[3]]]
<xarray.DataArray (x: 1, y: 3, time: 1)>
array([[[1.],
        [2.],
        [3.]]])
Coordinates:
  * x        (x) int32 1
  * y        (y) int32 1 2 3
  * time     (time) int32 1
[[[1.]
  [2.]
  [3.]]]


### Exercise

In Spyder, open a new Python script file and write the code to do the following in it:

1. Read in netCDF file "ch4-anthro_SOUTHAMERICA_2012.nc" and open as an `xarray.Dataset`. The file should be in the "data" subdirectory.
2. Extract the "flux" data variable
3. Extract the values from this data variable as a `numpy.array`
4. Calculate the mean of these flux values

### Bonus

 - Read netCDF file "ch4-fire_SOUTHAMERICA_2012.nc", extract values as before and calculate the sum for the September flux values.
*Hint: Check the dimensions of the flux variable to see which axis relates to time*

## Next Topic

When ready you can move onto the next topic:

### [Plotting using matplotlib](plotting.ipynb)

To view the introduction page containing the list of topics click [here](introduction.ipynb)